<a href="https://colab.research.google.com/github/markusfalkenhagen/aboutme/blob/main/Copy_of_recognize_anything_demo_1-working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **<font size=5> 🏷 Recognize Anything:** A Strong Image Tagging Model & **Tag2Text:** Guiding Vision-Language Model via Image Tagging </font>

Official PyTorch Implementation of the <a href="https://recognize-anything.github.io/">Recognize Anything Model (RAM)</a> and the <a href="https://tag2text.github.io/">Tag2Text Model</a>.

- RAM is an image tagging model, which can recognize any common category with high accuracy.
- Tag2Text is a vision-language model guided by tagging, which can support caption, retrieval and tagging.




In [ ]:
#@title Import dependencies
import ipywidgets as widgets
from IPython.display import clear_output, display, Image
import os

In [ ]:
#@title Clone the repository
!git clone https://github.com/xinyu1205/recognize-anything.git

%cd recognize-anything
!git checkout ec6b4241c5036e337a4543838deb9bff4990de97

Cloning into 'recognize-anything'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 527 (delta 203), reused 160 (delta 160), pack-reused 277
Receiving objects: 100% (527/527), 23.25 MiB | 23.02 MiB/s, done.
Resolving deltas: 100% (284/284), done.
/content/recognize-anything
Note: switching to 'ec6b4241c5036e337a4543838deb9bff4990de97'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at ec6b424 update


In [ ]:
#@title Install dependencies
!pip install timm transformers fairscale pycocoevalcap

clear_output()

In [ ]:
# Download checkpoints
model_widget = widgets.Dropdown(
    options=["RAM", "Tag2Text"],
    value="RAM",
    description="Select mdoel:"
)
display(model_widget)

Dropdown(description='Select mdoel:', options=('RAM', 'Tag2Text'), value='RAM')

In [ ]:
model = model_widget.value

In [ ]:
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")
    else:
        tag2text_weights_path = 'pretrained/tag2text_swin_14m.pth'
        if not os.path.exists(tag2text_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/tag2text_swin_14m.pth -O pretrained/tag2text_swin_14m.pth
        else:
            print("Tag2Text weights already downloaded!")

download_checkpoints(model)
print(model, 'weights are downloaded!')


You selected RAM
--2023-08-24 14:50:37--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 65.8.178.27, 65.8.178.118, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.27|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth [following]
--2023-08-24 14:50:37--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e6/78/e678f8565485a3f321b1180e4c7e1e18a89a9295028358eedffb98981b37e11a/15c729c793af28b9d107c69f85836a1356d76ea830d4714699fb62e55fcc08ed?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ram_swin_large_14m.pth%3B+filename%3D%22ram_swin_large_14m.p

In [ ]:
images_dir_widget = widgets.Text(value="images/demo", description="Images dir:")
display(images_dir_widget)

NameError: ignored

In [ ]:
images_dir = images_dir_widget.value

In [ ]:
image_files = [f"{images_dir}/{file}" for file in sorted(os.listdir(images_dir)) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
image_path = image_files[0]

# Create dropdown widget
image_dropdown = widgets.Dropdown(
    options=image_files,
    description='Select Image:',
)

# Create image preview widget
image_preview = widgets.Output()

# Define function to update image preview
def update_preview(change):
    global image_path
    image_path = change.new
    with image_preview:
        image_preview.clear_output()
        display(Image(filename=image_path, width=400))

# Set the initial image preview
with image_preview:
    display(Image(filename=image_files[0], width=400))

# Attach the update function to the dropdown
image_dropdown.observe(update_preview, names='value')

# Display the widgets
display(image_dropdown, image_preview)

In [ ]:
# Define the task and run inference
task_widget = widgets.Dropdown(
    options=["one image", "multiple images"],
    value="one image",
    description="Task:"
)
display(task_widget)

NameError: ignored

In [ ]:
task = task_widget.value

In [ ]:
# @title
print('You selected', model)
print('You selected', task)

def run_inference(model, task):
    if model == "Tag2Text" and task == "one image":
        !python inference_tag2text.py  --image {image_path} \
        --pretrained pretrained/tag2text_swin_14m.pth
    elif model == "Tag2Text" and task == "multiple images":
        !python batch_inference.py --image-dir {images_dir} \
        --pretrained pretrained/tag2text_swin_14m.pth --model-type tag2text
    elif model == "RAM" and task == "one image":
        !python inference_ram.py  --image {image_path} \
        --pretrained pretrained/ram_swin_large_14m.pth
    elif model == "RAM" and task == "multiple images":
        !python batch_inference.py --image-dir {images_dir} \
        --pretrained pretrained/ram_swin_large_14m.pth --model-type ram
    else:
        print('Invalid model or task')

run_inference(model, task)

You selected RAM
You selected multiple images
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 30524. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/cro

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
